<a href="https://colab.research.google.com/github/saimanit/CERTIFICATION/blob/main/gpt2_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.0 MB/s eta 0:00:00


In [2]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.8 MB/s eta 0:00:00


In [3]:
from PyPDF2 import PdfReader

In [4]:
from PyPDF2 import PdfReader
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [5]:
pdf_path = "Sai_Mani_Teja_Reddy_Alla.pdf"  # Adjust the path if necessary
reader = PdfReader(pdf_path)

resume_text = ""
for page in reader.pages:
    resume_text += page.extract_text() + "\n"

print("Extracted Resume Text:", resume_text)  # Print first 500 characters

Extracted Resume Text: Sai Mani Teja Reddy Alla
7165338659 manitejareddy16@gmail.com /githubsaimanit /hackerranksaimanit
Experience
DataSenseIT 09/2024 – PRESENT
Software Engineer Portland, Oregon
•Transformed system architecture by implementing microservices with Java Spring Boot on AWS, achieving a 25%
increase in performance while decreasing data retrieval speeds and latency by a significant 30%.
•Built and maintained CI/CD pipelines with Jenkins, Docker, and Kubernetes, reducing deployment times by 40% and
automating monitoring to decrease incident response times by 30%.
•Developed responsive user interfaces with React, Redux, and JavaScript, improving user engagement by 15%, and
optimized database performance using AWS Athena, MySQL, and PostgreSQL.
•Devised comprehensive security measures through implementation of Spring Security and JWT while integrating
advanced testing strategies; delivered a notable 15% decline in post-release defect occurrences for improved software
stabilit

In [6]:
# ✅ Step 3: Initialize Tokenizer & Model (GPT-2)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# ✅ Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ✅ Step 4: Process & Tokenize Resume Text
encoded_resume = tokenizer(
    resume_text,
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors="pt",
    add_special_tokens=True  # Ensures the model understands sentence structure
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
# ✅ Step 5: Define Custom Dataset Class
class ResumeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = item['input_ids'].clone()  # Labels are the same as input_ids for language modeling
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

resume_dataset = ResumeDataset(encoded_resume)
print("✅ Resume dataset created successfully!")


✅ Resume dataset created successfully!


In [8]:
# ✅ Step 6: Detect & Set Device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to GPU if available

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [9]:
# ✅ Step 7: Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=5,  # Reduce training time
    logging_dir='./logs',
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
# ✅ Step 8: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=resume_dataset,
    eval_dataset=resume_dataset  # Using same dataset for simplicity
)



In [11]:
# ✅ Step 9: Train the Model
trainer.train()
print("✅ Model trained successfully!")



wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saimanit (saimanit-university-at-buffalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


<ipython-input-7-c92b101ff728>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,3.712634
2,No log,3.526370
3,No log,3.395543
4,No log,3.308277
5,No log,3.264617


<ipython-input-7-c92b101ff728>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


✅ Model trained successfully!


In [12]:
# ✅ Step 10: Define Prompt for Text Generation
prompt = (
    "As a software engineer, I have expertise in cloud computing, data engineering, "
    "and software development. My experience includes designing and deploying enterprise-scale "
    "applications using AWS, Java, Python, and SQL. "
    "I have successfully built RESTful APIs with Spring Boot and optimized database queries for performance. "
    "Additionally, I have led projects integrating machine learning models into production environments."
)




In [13]:
# ✅ Step 11: Tokenize Prompt & Move to Device
input_prompt = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
input_prompt = {key: val.to(device) for key, val in input_prompt.items()}  # Move tensors to same device



In [14]:
# ✅ Step 12: Generate Text Using Trained Model
generated_ids = model.generate(
    input_ids=input_prompt['input_ids'],
    attention_mask=input_prompt['attention_mask'],
    max_new_tokens=150,  # Control length
    temperature=0.1,  # Extremely low temp for deterministic, factual responses
    top_k=20,  # Lower randomness by selecting only the top tokens
    top_p=0.80,  # Further limit probability mass for predictable output
    repetition_penalty=4.0,  # Stronger penalty to stop hallucinations & repetition
    num_beams=5,  # Higher beam search for structured, refined results
    early_stopping=True,  # Ensures the output doesn’t go off-track
    no_repeat_ngram_size=3  # Prevents repeating 3-word phrases
)







/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [15]:
# ✅ Step 13: Decode & Print Generated Text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("\n📝 Generated Resume-Based Text:\n", generated_text)


📝 Generated Resume-Based Text:
 As a software engineer, I have expertise in cloud computing, data engineering, and software development. My experience includes designing and deploying enterprise-scale applications using AWS, Java, Python, and SQL. I have successfully built RESTful APIs with Spring Boot and optimized database queries for performance. Additionally, I have led projects integrating machine learning models into production environments.In addition to my technical background, I am an active member of the Open Source Software Foundation (OSSF), which promotes open source technologies such as distributed systems programming languages like C++, Ruby, and JavaScript.

I'm currently working on a project that will allow me to use Google Cloud Platform (GCP) instead of Amazon Web Services (AWS). The goal is to make it easier for developers to deploy their apps across multiple platforms without having to rely on third-party libraries or frameworks. In order to do this, I'll be able 